In [1]:
import pandas as pd
import numpy as np
import os,sys

pd.options.display.max_rows = 10
enter = lambda :print('----------------------------------------------')

# 7. 데이터 정제 및 준비

## 7.1 누락된 데이터 처리하기  
- Pandas의 설계목적중 하나는 누락된 데이터를 쉽게 처리할수 있도록 하는것,  
- 모든 기술 통계는 누락된 데이터를 배제하고 처리한다.

In [2]:
string_data = pd.Series(['aardvark','artichoke',np.nan,'avocado'])
string_data;string_data.isnull()

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

0    False
1    False
2     True
3    False
dtype: bool

In [3]:
#python 내장 None도 NA값으로 취급됨
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 7.1.1 누락된 데이터 골라내기  
- isnull 사용하거나 불리언 색인을 사용해 직접 제거할 수도 있으나,  
- dropna를 사용하면 유용함

In [4]:
from numpy import nan as NA
data= pd.Series([1,NA,3.5,NA,7])
data
enter()
data.dropna()

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

----------------------------------------------


0    1.0
2    3.5
4    7.0
dtype: float64

데이터 프래임의 경우 복잡함, NA를 하나라도 포함한 행렬 다날리거나, 행만 날리거나 할수 있음  
기본적으로 NA가 하나라도 포함된 Row를 날림

In [5]:
data = pd.DataFrame([[1., 6.5, 3.],
                     [1., NA, NA],
                     [NA, NA, NA],
                     [NA, 6.5, 3.]])
data
enter()
data.dropna()

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


----------------------------------------------


,0,1,2
0,1.0,6.5,3.0


how = 'all' 옵션을 넣으면 모두 NA인 놈만 날림

In [6]:
data.dropna(how='all')

#axis 옵션도 먹음
data[4]=NA
data;enter()
data.dropna(axis=1,how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


----------------------------------------------


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


**값이 x개 이상인 로우만 보고싶다면 thresh 인자에 값을 넘기면 됨**

In [7]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1]=NA
df.iloc[:2,2]=NA
df;enter()
df.dropna();enter()
df.dropna(thresh=2)

,0,1,2
0,-0.167487,NaN,NaN
1,-0.453423,NaN,NaN
2,-0.554527,NaN,-0.195489
3,0.861427,NaN,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


----------------------------------------------


,0,1,2
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


----------------------------------------------


,0,1,2
2,-0.554527,NaN,-0.195489
3,0.861427,NaN,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


### 7.1.2 결측치 채우기  
fillna 사용

In [8]:
df;enter()
df.fillna(0)

,0,1,2
0,-0.167487,NaN,NaN
1,-0.453423,NaN,NaN
2,-0.554527,NaN,-0.195489
3,0.861427,NaN,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


----------------------------------------------


,0,1,2
0,-0.167487,0.000000,0.000000
1,-0.453423,0.000000,0.000000
2,-0.554527,0.000000,-0.195489
3,0.861427,0.000000,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


In [9]:
#컬럼마다 다른 값을 채우기
df.fillna({1:0.5,2:0})

,0,1,2
0,-0.167487,0.500000,0.000000
1,-0.453423,0.500000,0.000000
2,-0.554527,0.500000,-0.195489
3,0.861427,0.500000,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


fillna는 기본적으로 새객체를 반환하지만 기존객체를 변경할수도 있음 (inplace옵션)

In [10]:
df;enter()
df.fillna(0);df;enter()
_ = df.fillna(0,inplace=True)

,0,1,2
0,-0.167487,NaN,NaN
1,-0.453423,NaN,NaN
2,-0.554527,NaN,-0.195489
3,0.861427,NaN,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


----------------------------------------------


,0,1,2
0,-0.167487,0.000000,0.000000
1,-0.453423,0.000000,0.000000
2,-0.554527,0.000000,-0.195489
3,0.861427,0.000000,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


,0,1,2
0,-0.167487,NaN,NaN
1,-0.453423,NaN,NaN
2,-0.554527,NaN,-0.195489
3,0.861427,NaN,-0.430741
4,3.403501,-0.130103,-1.011388
5,-1.362698,0.153033,-1.293492
6,-0.059562,0.167445,0.430381


----------------------------------------------


보간 메서드를 그대로 사용가능

In [11]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:,1] = NA
df.iloc[:4,2] = NA
df;enter()
df.fillna(method='ffill')
df.fillna(method='ffill',limit=2)
df.fillna(method='bfill')

,0,1,2
0,0.963494,-2.194161,NaN
1,0.597720,-0.286258,NaN
2,-0.561197,NaN,NaN
3,-1.491386,NaN,NaN
4,0.417720,NaN,-0.883423
5,0.814235,NaN,-0.491306


----------------------------------------------


,0,1,2
0,0.963494,-2.194161,NaN
1,0.597720,-0.286258,NaN
2,-0.561197,-0.286258,NaN
3,-1.491386,-0.286258,NaN
4,0.417720,-0.286258,-0.883423
5,0.814235,-0.286258,-0.491306


,0,1,2
0,0.963494,-2.194161,NaN
1,0.597720,-0.286258,NaN
2,-0.561197,-0.286258,NaN
3,-1.491386,-0.286258,NaN
4,0.417720,NaN,-0.883423
5,0.814235,NaN,-0.491306


,0,1,2
0,0.963494,-2.194161,-0.883423
1,0.597720,-0.286258,-0.883423
2,-0.561197,NaN,-0.883423
3,-1.491386,NaN,-0.883423
4,0.417720,NaN,-0.883423
5,0.814235,NaN,-0.491306


fillna를 조금 응용하면 Series의 평균값이나 중간값을 전달할 수 도있음

In [12]:
data = pd.Series([1,NA,3.5,NA,7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 데이터 변형

### 7.2.1 중복 제거하기

In [13]:
data = pd.DataFrame({'k1':['one','two']*3 + ['two'],
                    'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


duplicated 메서드는 각 로우가 중복인지 아닌자 알려주는 불리언 Series를 반환한다.

In [14]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

drop_duplicates 는 중복이 아닌 DF를 반환한다.

In [15]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


duplicated 와 drop_duplicates는 기본적으로 모든 컬럼에 적용되고,  
중복을 찾아내기 위한 부분합을 따로 지정해 줄수도 있음

In [16]:
data['v1']=range(7)
data
data.drop_duplicates(['k1'])
print('처음발견된 값을 유지함')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


,k1,k2,v1
0,one,1,0
1,two,1,1


처음발견된 값을 유지함


keep='last'옵션을 주면 마지막 값을 반환함

In [17]:
data.drop_duplicates(['k1'],keep='last')

,k1,k2,v1
4,one,3,4
6,two,4,6


### 7.2.2 함수나 매핑을 이용해서 데이터 변형하기

In [18]:
data = pd.DataFrame({'food':['bacon','pulled pork','bacon',
                            'Pastrami','corned beef','Bacon',
                            'pastrami','honey ham','nova lox'],
                    'ounces':[4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


육류가 어떤 동물의 고기인지 알려줄 수 있는 컬럼을 추가한다고 가정하고 사전작성

In [19]:
meat_to_animal = {
    'bacon':'pig',
    'pulled pork':'pig',
    'pastrami':'cow',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salmon'
}

Series의 map 메서드는 사전류의 객체나 함수를 받을수 있는데 대소문자가 섞여있으므로 str.lower로 모두 소문자로 변경

In [20]:
lowercased = data['food'].str.lower()
lowercased
#lowercased.str.lower()

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [21]:
data['animal'] = lowercased.map(meat_to_animal) ### 딕셔너리 넘길수 이따!!
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [22]:
#함수로 넘길경우
data['food'].map(lambda x:meat_to_animal[str(x).lower()] )

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.2.3 값 치환하기  
- map 메서드를 사용해서 객체안의 부분집합을 변경했다면,  
- replace 메서드는 조금더 간단하고 유연한 방법을 제공한다.

In [23]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999는 누락된 데이터를 나타내기 위해 사용된 값임, replace를 이용해 NA로 치환

In [24]:
data.replace(-999,np.nan) #inplace=True 옵션을 주면 원본데이터가 변경됨

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

**여러개의 값을 한번에 치환하는 방법들**

In [25]:
data.replace([-999,-1000],np.nan)
data.replace([-999,-1000],[np.nan,0])
data.replace({-999:np.nan,
             -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.2.4 축의 색인 이름 바꾸기

In [26]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index=map(str.title,['Ohio','colorado','new york']),
                   columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [27]:
#Index 객체에도 map 메서드가 있음
transform = lambda x:x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [28]:
#변경된 축이름을 index에 바로 대입가능
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


원래 객체를 변경하지 않고 새로운 객체를 생성하려면 rename 메서드를 이용

In [29]:
data.rename(index=str.title,columns=str.upper)
data

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename 메서드는 사전형식의 객체를 이용해서 **축 이름중 일부만** 변경하는것도 가능함

In [30]:
data.rename(index={'OHIO':'INDIANA'},
           columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename 메서드를 사용하면 DataFrame을 직접 복사해서 index와 columns 속성을 갱신할 필요없이 바로 변경가능함  
원본 데이터를 바로 변경하려면 inplace 옵션

In [31]:
data
data.rename(index={'OHIO':'INDIANA'},
           columns={'three':'peekaboo'},inplace=True)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 개별화와 양자화  
연속성 데이터를 개별로 분할하거나, 그룹별로 나누기

In [32]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]

pandas.cut 함수 이용해서 나누기

In [33]:
bins = [18,25,35,60,100]
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

Categorical 이라는 특수 객체 반환됨, 이 객체는 codes 속성에 있는 ages 데이터에 대한 카테고리 이름을 categories 배열에 담고있음

In [34]:
cats.codes
cats.categories
cats.value_counts()

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

↑[]값은 포함안하고 ()값은 포함임, right=False를 넘기면 중괄호 대신 대괄호 쪽이 포함되지 않도록 바꿀수 있음

In [35]:
pd.cut(ages,[18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

labels 옵션을 주면 그룹의 이름을 직접 넘겨줄수도 있음

In [36]:
group_names = ['Youth','YoungAdult','MiddleAged','Senior']
pd.cut(ages,bins,labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

만약 명시적으로 경계값을 안넘기고 그룹 갯수만 넘기면, 최소,최대값 기준으로 균등길이를 자동 계산함

In [37]:
data = np.random.rand(20)
pd.cut(data,4,precision=2) #precision 옵션으로 소수점 아래 2자리 까지로 제한

[(0.76, 1.0], (0.27, 0.51], (0.27, 0.51], (0.51, 0.76], (0.27, 0.51], ..., (0.76, 1.0], (0.76, 1.0], (0.76, 1.0], (0.027, 0.27], (0.27, 0.51]]
Length: 20
Categories (4, interval[float64]): [(0.027, 0.27] < (0.27, 0.51] < (0.51, 0.76] < (0.76, 1.0]]

이처럼 그룹을 자동계산하면 데이터의 산포때문에 그룹별로 할당되는 갯수가 달라질수 있는데,  
qcut 함수는 표본의 변위치를 기반으로 나누기때문에 적당히 같은 크기로 나눠짐

In [38]:
data = np.random.randn(1000)
cat1 = pd.cut(data,4)
cat2 = pd.qcut(data,4)
cat1.value_counts()
cat2.value_counts()

(-3.115, -1.465]     68
(-1.465, 0.179]     497
(0.179, 1.823]      408
(1.823, 3.467]       27
dtype: int64

(-3.109, -0.628]    250
(-0.628, 0.0361]    250
(0.0361, 0.718]     250
(0.718, 3.467]      250
dtype: int64

In [39]:
#변위를 직접 지정해줄수도 있음 0~1
cat3 = pd.qcut(data,[0,0.1,0.5,0.8,1])
cat3
cat3.value_counts()

[(0.0361, 0.944], (0.944, 3.467], (0.944, 3.467], (0.0361, 0.944], (-1.257, 0.0361], ..., (0.0361, 0.944], (-1.257, 0.0361], (-1.257, 0.0361], (-1.257, 0.0361], (-1.257, 0.0361]]
Length: 1000
Categories (4, interval[float64]): [(-3.109, -1.257] < (-1.257, 0.0361] < (0.0361, 0.944] < (0.944, 3.467]]

(-3.109, -1.257]    100
(-1.257, 0.0361]    400
(0.0361, 0.944]     300
(0.944, 3.467]      200
dtype: int64

### 7.2.6 특잇값을 찾고 제외하기

In [40]:
data = pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.049788,-0.005944,0.037633,0.007987
std,1.001402,1.017269,0.998422,0.995412
min,-3.126814,-3.222196,-3.011118,-3.278490
25%,-0.709399,-0.698035,-0.644987,-0.652603
50%,-0.054548,-0.032222,0.033613,0.017160
75%,0.588872,0.701168,0.692382,0.664928
max,2.893291,2.928052,3.283010,2.860651


In [41]:
#한 컬럼에서 절대값이 3을 넘는 놈 찾기
col = data[3]
col[np.abs(col) > 3]

766   -3.248164
883   -3.278490
Name: 3, dtype: float64

In [42]:
(np.abs(data) > 3).any(axis=1)
#절대값이 3을 넘는놈을 포함한 로우 선택
data[(np.abs(data)>3).any(axis=1)]

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

,0,1,2,3
52,-3.126814,0.561537,0.232197,1.981397
298,-0.015019,-3.222196,0.254146,0.183172
351,-0.128905,-1.060484,3.030806,-0.743858
520,1.033179,-3.015361,-0.150368,0.890503
620,-0.577259,0.116161,-3.011118,-0.224922
766,-0.482134,-1.014476,-0.708122,-3.248164
774,1.084034,-0.105708,3.283010,0.186492
883,-0.488138,-0.593248,-0.543877,-3.278490


절대값이 3이 넘는 놈들 제한하기

In [43]:
data[(np.abs(data)>3)] = np.sign(data) *3
# 슬라이싱 한놈은 원본의 투영이니까 이런게 가능한듯
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.049661,-0.005707,0.037330,0.008514
std,1.001020,1.016545,0.997415,0.993748
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.709399,-0.698035,-0.644987,-0.652603
50%,-0.054548,-0.032222,0.033613,0.017160
75%,0.588872,0.701168,0.692382,0.664928
max,2.893291,2.928052,3.000000,2.860651


### 7.2.7 치환과 임의 샘플링  
- np.random.permutation 함수를 이용하면 로우 순서를 임의로 재배치할수 있음

In [44]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler = np.random.permutation(5)
df
sampler

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


array([2, 0, 1, 3, 4])

섞인 순서는 iloc 기반 색인이나, take 함수를 통해 사용할수 있음

In [45]:
df.iloc[sampler]
df.take(sampler)

,0,1,2,3
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


,0,1,2,3
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


In [46]:
a = pd.DataFrame(np.arange(12).reshape((4,3)))
a
column_sampler = np.random.permutation(a.shape[1])
row_sampler = np.random.permutation(a.shape[0])
column_sampler;row_sampler

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


array([1, 0, 2])

array([0, 2, 1, 3])

In [47]:
a[column_sampler]
a.iloc[row_sampler]
a.iloc[row_sampler,column_sampler]

,1,0,2
0,1,0,2
1,4,3,5
2,7,6,8
3,10,9,11


,0,1,2
0,0,1,2
2,6,7,8
1,3,4,5
3,9,10,11


,1,0,2
0,1,0,2
2,7,6,8
1,4,3,5
3,10,9,11


치환 없이 일부만 **선택**하려면 sample 메서드 사용

In [48]:
df.sample(n=3)
df.sample(n=2,axis=1)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11


,1,3
0,1,3
1,5,7
2,9,11
3,13,15
4,17,19


반복 선택을 허용하면서 치환을 통해 생성하려면 sample에 replace = True 옵션을 줌

In [49]:
choices = pd.Series([5,7,-1,6,4])
choices
draw = choices.sample(n=10,replace=True)
draw

0    5
1    7
2   -1
3    6
4    4
dtype: int64

0    5
3    6
4    4
3    6
4    4
1    7
3    6
4    4
2   -1
4    4
dtype: int64

### 7.2.8 표시자/더미 변수 계산하기  
- 통계모델이나 머신러닝에서 사용하기 위한 변환법으로, 분류값을 더미나 표시자로 변경  
- One-hot 인코딩 말하는듯  
- get_dummies 함수 이용

In [50]:
df = pd.DataFrame({'key':['b','b','a','c','a','b'],
                  'data1':range(6)})
df
pd.get_dummies(df['key'])

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


표시자 행렬에 접두어를 추가해서 다른데이터와 병합하고 싶은 경우 prefix 인자 이용

In [51]:
dummies=pd.get_dummies(df['key'],prefix='key')
dummies
df['data1']
df[['data1']]
df[['data1']].join(dummies)
dummies.join(df[['data1']])

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


0    0
1    1
2    2
3    3
4    4
5    5
Name: data1, dtype: int64

,data1
0,0
1,1
2,2
3,3
4,4
5,5


,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


,key_a,key_b,key_c,data1
0,0,1,0,0
1,0,1,0,1
2,1,0,0,2
3,0,0,1,3
4,1,0,0,4
5,0,1,0,5


**DataFrame의 한 로우가 여러 카테고리에 속하는 경우**14.2에서 자세히..

In [52]:
mnames=['movie_id','title','genres']
movies = pd.read_csv('datasets/movielens/movies.dat',sep='::',
                      header=None,engine='python',names=mnames)
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


먼저 장르 종류를 얻기위해 유일한 값을 얻어냄, |로 겹쳐있으니 split하고

In [53]:
# all_genres = []
# for i in range(len(movies)):
#     dat = movies['genres'][i].split('|')
#     for gen in dat:
#         if gen not in all_genres: all_genres.append(gen)

all_genres = []
for gen in movies['genres']:
    all_genres.extend(gen.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

표시자 행렬 생성

In [54]:
zero_mat = np.zeros((len(movies),len(genres)))
dummies = pd.DataFrame(zero_mat,columns = genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


각 영화를 순회하면서 dummies의 각 로우항목을 1로 설정

In [55]:
gen = movies.genres[0].split('|')
gen

['Animation', "Children's", 'Comedy']

In [56]:
dummies.columns.get_indexer(gen)

array([0, 1, 2], dtype=int64)

In [57]:
for i,gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i,indices]=1

In [58]:
# for i in range(len(movies)):
#     gen = movies.genres[i].split('|')
#     dummies.iloc[i][gen] = 1

In [59]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.head()

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


get_dummies와 cut을 조합하여 통계 어플리케이션에 사용하는예

In [60]:
np.random.seed(12345)
values = np.random.rand(10)
values

bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values,bins))

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

## 7.3 문자열 다루기